In [3]:
import os
import arcpy
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [2]:
gdb = r"Q:\projects\Mwlrs\AEB_Prioritization_Tool\data\AEB_Analysis.gdb"
arcpy.env.workspace= gdb

In [11]:
# create a list of ecosection/stream intersections
fc= 'intersect_stream_ecosection'

fields = ["BLUE_LINE_KEY", "SUMMER_SNTVTY", "WINTER_SNTVTY", "SHAPE@LENGTH"]

data = []

rowcount= 0
with arcpy.da.SearchCursor(fc, fields) as cursor:
    for row in cursor:
        blue_key= row[0]
        summer_sens = row[1]
        winter_sens = row[2]
        length_m = row[3] #in meters
        data.append([blue_key, summer_sens, winter_sens, length_km])

        rowcount += 1

In [12]:
# create a df for raw data
df_raw = pd.DataFrame(data, columns=["BLUE_LINE_KEY", "SUMMER_SNTVTY", "WINTER_SNTVTY", "LENGTH_M"])
df_raw.head()

,BLUE_LINE_KEY,SUMMER_SNTVTY,WINTER_SNTVTY,LENGTH_M
0,359547255,Not Sensitive,Sensitive,1.616603
1,359251732,Not Sensitive,Sensitive,1.616603
2,359385888,Not Sensitive,Sensitive,1.616603
3,359349985,Not Sensitive,Sensitive,1.616603
4,359137923,Not Sensitive,Sensitive,1.616603


In [23]:
# calulcate the total stream length by ecosection
df_tot= df_raw.groupby(["BLUE_LINE_KEY"])["LENGTH_M"].sum().reset_index()

df_tot.rename(columns={"LENGTH_M": "TOTAL_LENGTH_M"}, inplace=True)

df_tot.head()

,BLUE_LINE_KEY,TOTAL_LENGTH_M
0,354087611,6.466414
1,354087612,9.699621
2,354087613,6.466414
3,354087614,12.932828
4,354087615,4.849810


In [51]:
#calulcate SUMMER sensitvity stats

summer_length_sum = df_raw.groupby(["BLUE_LINE_KEY", "SUMMER_SNTVTY"])["LENGTH_M"].sum().reset_index()

# renaming columns for clarity
summer_length_sum.rename(columns={"LENGTH_M": "SUMMER_LENGTH_SUM"}, inplace=True)



# add total lengths
df_summer_sum= pd.merge(
   summer_length_sum,
   df_tot,
   how='left',
   on= 'BLUE_LINE_KEY'
)

# renaming sensitivty vlalues for pivoting
df_summer_sum['SUMMER_SNTVTY'] = df_summer_sum['SUMMER_SNTVTY'].replace({
    'Not Sensitive': 'NOT_SNSTV',
    'Sensitive': 'SNSTV'
})

# calculate percentage
df_summer_sum['SUMMER_PERCENT'] = round((df_summer_sum['SUMMER_LENGTH_SUM'] /df_summer_sum['TOTAL_LENGTH_M'] )*100, 0)

# Pivot the table
df_summer_sum_pv = df_summer_sum.pivot_table(
    index=["BLUE_LINE_KEY"],
    columns='SUMMER_SNTVTY',
    values=['SUMMER_PERCENT'],
    fill_value=0
)

df_summer_sum_pv.columns = [
    f"{col[0]}_{col[1]}" if col[1] else col[0] 
    for col in df_summer_sum_pv.columns
]

df_summer_sum_pv.reset_index(inplace=True)

df_summer_sum_pv.head()


,BLUE_LINE_KEY,SUMMER_PERCENT_NOT_SNSTV,SUMMER_PERCENT_SNSTV
0,354087611,0,100
1,354087612,0,100
2,354087613,0,100
3,354087614,0,100
4,354087615,0,100


In [53]:
#calulcate WINTER sensitvity stats

winter_length_sum = df_raw.groupby(["BLUE_LINE_KEY", "WINTER_SNTVTY"])["LENGTH_M"].sum().reset_index()

# renaming columns for clarity
winter_length_sum.rename(columns={"LENGTH_M": "WINTER_LENGTH_SUM"}, inplace=True)


# add total lengths
df_winter_sum= pd.merge(
   winter_length_sum,
   df_tot,
   how='left',
   on= 'BLUE_LINE_KEY'
)

# renaming sensitivty vlalues for pivoting
df_winter_sum['WINTER_SNTVTY'] = df_winter_sum['WINTER_SNTVTY'].replace({
    'Not Sensitive': 'NOT_SNSTV',
    'Sensitive': 'SNSTV'
})

# calculate percentage
df_winter_sum['WINTER_PERCENT'] = round((df_winter_sum['WINTER_LENGTH_SUM'] /df_winter_sum['TOTAL_LENGTH_M'] )*100, 0)

# Pivot the table
df_winter_sum_pv = df_winter_sum.pivot_table(
    index=["BLUE_LINE_KEY"],
    columns='WINTER_SNTVTY',
    values=['WINTER_PERCENT'],
    fill_value=0
)

df_winter_sum_pv.columns = [
    f"{col[0]}_{col[1]}" if col[1] else col[0] 
    for col in df_winter_sum_pv.columns
]

df_winter_sum_pv.reset_index(inplace=True)

df_winter_sum_pv.head()


,BLUE_LINE_KEY,WINTER_PERCENT_NOT_SNSTV,WINTER_PERCENT_SNSTV
0,354087611,100,0
1,354087612,100,0
2,354087613,100,0
3,354087614,100,0
4,354087615,100,0


In [56]:
# join SUMMER and WINTER stats
df_stats= pd.merge(
   df_summer_sum_pv,
   df_winter_sum_pv,
   how='inner',
   on= 'BLUE_LINE_KEY'
)

df_stats.head()

,BLUE_LINE_KEY,SUMMER_PERCENT_NOT_SNSTV,SUMMER_PERCENT_SNSTV,WINTER_PERCENT_NOT_SNSTV,WINTER_PERCENT_SNSTV
0,354087611,0,100,100,0
1,354087612,0,100,100,0
2,354087613,0,100,100,0
3,354087614,0,100,100,0
4,354087615,0,100,100,0


In [65]:
df_t= df_stats[df_stats['BLUE_LINE_KEY']==360883982]
df_t

,BLUE_LINE_KEY,SUMMER_PERCENT_NOT_SNSTV,SUMMER_PERCENT_SNSTV,WINTER_PERCENT_NOT_SNSTV,WINTER_PERCENT_SNSTV
1778484,360883982,39,61,55,45


In [66]:
csv_file= r'Q:\dss_workarea\mlabiadh\workspace\20241125_AEB\data\stats.csv'
#df_stats.to_csv(csv_file)

In [68]:
# load the streams featureclass as esri geodataframe
strm_sedf= pd.DataFrame.spatial.from_featureclass(
    f"{gdb}/streams_assessUnits_ecosections"
)

strm_sedf.head()

,OBJECTID,LINEAR_FEATURE_ID,WATERSHED_GROUP_ID,EDGE_TYPE,BLUE_LINE_KEY,WATERSHED_KEY,FWA_WATERSHED_CODE,LOCAL_WATERSHED_CODE,WATERSHED_GROUP_CODE,DOWNSTREAM_ROUTE_MEASURE,...,ASSESSMENT_UNIT_NAME,ASSESSMENT_UNIT_GROUP,MAJOR_STREAM_FLAG,PARENT_ECO,ECOSECTION_NAME,SUMMER_SNTVTY,WINTER_SNTVTY,STREAM_SUMMER_SNTVTY,STREAM_WINTER_SNTVTY,SHAPE
0,1,213022948,214,1000,360785318,360785318,600-392439-581117-849075-826060-000000-000000-...,600-392439-581117-849075-826060-000000-000000-...,TUYR,129.743839,...,<NA>,TUYR,N,BMP,Stikine Plateau,Not Sensitive,Sensitive,Not Sensitive,Sensitive,"{""hasZ"": true, ""paths"": [[[706013.8650000002, ..."
1,2,213018919,214,1200,360625854,360625854,600-392439-581117-941274-380896-000000-000000-...,600-392439-581117-941274-380896-000000-000000-...,TUYR,268.405694,...,<NA>,TUYR,N,BMP,Stikine Plateau,Not Sensitive,Sensitive,Not Sensitive,Sensitive,"{""hasZ"": true, ""paths"": [[[706871.3959999997, ..."
2,3,213054054,214,1200,360397908,360397908,600-392439-160034-761784-354968-000000-000000-...,600-392439-160034-761784-354968-000000-000000-...,TUYR,316.198887,...,Little Tuya River,TUYR,N,BMP,Stikine Plateau,Not Sensitive,Sensitive,Not Sensitive,Sensitive,"{""hasZ"": true, ""paths"": [[[697013.0629999992, ..."
3,4,213055363,214,1200,360819807,360819807,600-392439-431789-518848-421403-265347-000000-...,600-392439-431789-518848-421403-265347-359691-...,TUYR,2536.570403,...,<NA>,TUYR,N,BMP,Stikine Plateau,Not Sensitive,Sensitive,Not Sensitive,Sensitive,"{""hasZ"": true, ""paths"": [[[709190.4979999997, ..."
4,5,213060324,214,1050,360825135,360825135,600-392439-268767-432078-599005-000000-000000-...,600-392439-268767-432078-599005-660645-000000-...,TUYR,1840.297264,...,<NA>,TUYR,N,BMP,Stikine Plateau,Not Sensitive,Sensitive,Not Sensitive,Sensitive,"{""hasZ"": true, ""paths"": [[[717521.2960000001, ..."


In [70]:
# join stats to the esri spatial dataframe

df_final= pd.merge(
    strm_sedf,
    df_stats,
    how= 'left',
    on= 'BLUE_LINE_KEY'
)  

# fill na in new cols
cols = ['SUMMER_PERCENT_NOT_SNSTV', 'SUMMER_PERCENT_SNSTV', 'WINTER_PERCENT_NOT_SNSTV', 'WINTER_PERCENT_SNSTV']
df_final[columns_to_fill] = df_final[columns_to_fill].fillna(0).astype(int)

df_final.head()


,OBJECTID,LINEAR_FEATURE_ID,WATERSHED_GROUP_ID,EDGE_TYPE,BLUE_LINE_KEY,WATERSHED_KEY,FWA_WATERSHED_CODE,LOCAL_WATERSHED_CODE,WATERSHED_GROUP_CODE,DOWNSTREAM_ROUTE_MEASURE,...,ECOSECTION_NAME,SUMMER_SNTVTY,WINTER_SNTVTY,STREAM_SUMMER_SNTVTY,STREAM_WINTER_SNTVTY,SHAPE,SUMMER_PERCENT_NOT_SNSTV,SUMMER_PERCENT_SNSTV,WINTER_PERCENT_NOT_SNSTV,WINTER_PERCENT_SNSTV
0,1,213022948,214,1000,360785318,360785318,600-392439-581117-849075-826060-000000-000000-...,600-392439-581117-849075-826060-000000-000000-...,TUYR,129.743839,...,Stikine Plateau,Not Sensitive,Sensitive,Not Sensitive,Sensitive,"{""hasZ"": true, ""paths"": [[[706013.8650000002, ...",100.0,0.0,0.0,100.0
1,2,213018919,214,1200,360625854,360625854,600-392439-581117-941274-380896-000000-000000-...,600-392439-581117-941274-380896-000000-000000-...,TUYR,268.405694,...,Stikine Plateau,Not Sensitive,Sensitive,Not Sensitive,Sensitive,"{""hasZ"": true, ""paths"": [[[706871.3959999997, ...",100.0,0.0,0.0,100.0
2,3,213054054,214,1200,360397908,360397908,600-392439-160034-761784-354968-000000-000000-...,600-392439-160034-761784-354968-000000-000000-...,TUYR,316.198887,...,Stikine Plateau,Not Sensitive,Sensitive,Not Sensitive,Sensitive,"{""hasZ"": true, ""paths"": [[[697013.0629999992, ...",100.0,0.0,0.0,100.0
3,4,213055363,214,1200,360819807,360819807,600-392439-431789-518848-421403-265347-000000-...,600-392439-431789-518848-421403-265347-359691-...,TUYR,2536.570403,...,Stikine Plateau,Not Sensitive,Sensitive,Not Sensitive,Sensitive,"{""hasZ"": true, ""paths"": [[[709190.4979999997, ...",100.0,0.0,0.0,100.0
4,5,213060324,214,1050,360825135,360825135,600-392439-268767-432078-599005-000000-000000-...,600-392439-268767-432078-599005-660645-000000-...,TUYR,1840.297264,...,Stikine Plateau,Not Sensitive,Sensitive,Not Sensitive,Sensitive,"{""hasZ"": true, ""paths"": [[[717521.2960000001, ...",100.0,0.0,0.0,100.0


In [ ]:
# save the new dataset to gdb
#df_final.spatial.to_featureclass(location=f"{gdb}/streams_drought_sensitivity")